In [34]:
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn import linear_model
from statsmodels.iolib.smpickle import save_pickle
import pandas as pd 

data = [{'periodo': "2023-02-28",
        "carne_producida": "0",
        "carneros": "123",
        "corderos": "123",
        "direccion_viento": 245,
        "finura_lana": "0",
        "humedad": 21,
        "lana_producida": "123",
        "localidad": "Gaimán",
        "mm_lluvia": 12,
        "muertes": "0",
        "ovejas": "123",
        "pariciones": "0",
        "rinde_lana": "0",
        "temperatura_maxima": 31.02,
        "temperatura_minima": 31.02,
        "velocidad_viento": 8.03},
        {'periodo': "2023-02-28",
        "carne_producida": "0",
        "carneros": "123",
        "corderos": "123",
        "direccion_viento": 245,
        "finura_lana": "0",
        "humedad": 21,
        "lana_producida": "123",
        "localidad": "Gaimán",
        "mm_lluvia": 12,
        "muertes": "0",
        "ovejas": "123",
        "pariciones": "0",
        "rinde_lana": "0",
        "temperatura_maxima": 31.02,
        "temperatura_minima": 31.02,
        "velocidad_viento": 8.03}]

_lags=1
df = pd.DataFrame(data)
df['total_ovinos'] = df.apply(lambda row: int(row.ovejas) + int(row.carneros) + int(row.corderos), axis=1)
df.rename(columns={'periodo':'fecha', 
                   'corderos': 'corderoas',
                   'mm_lluvia':'cant_lluvia',
                   'lana_producida': 'kilos_lana',
                   'finura_lana': 'finura',
                   'rinde_lana': 'rinde'}, inplace=True)

df = df[['fecha',"cant_lluvia", "total_ovinos",'kilos_lana','ovejas','corderoas', 'finura', 'rinde']]

df.cant_lluvia = df.cant_lluvia.astype('float')
df.finura = df.finura.astype('float')
df.rinde = df.rinde.astype('float')
df.total_ovinos = df.total_ovinos.astype('int64')
df.kilos_lana = df.kilos_lana.astype('int64')
df.total_ovinos = df.total_ovinos.astype('int64')

df.ovejas = df.ovejas.astype('int64')
df.corderoas = df.corderoas.astype('int64')
dataset = df


#dataset = pd.read_csv('pepita_data.csv')
print(dataset.columns)
#------------ LANA
X_train = dataset[["cant_lluvia", "total_ovinos"]]
y_train = dataset[['kilos_lana']]
model_lana = linear_model.LinearRegression()
model_lana.fit(X_train, y_train)
save_pickle(model_lana,'___modelo_prediccion_lana')

#-------------- CORDEROS
X_train = dataset[["cant_lluvia", "ovejas"]]
y_train = dataset[['corderoas']]
model_corderos = linear_model.LinearRegression()
model_corderos.fit(X_train, y_train)
save_pickle(model_corderos,'___modelo_prediccion_corderos')

#-------------- FINURA
datos_train = dataset[['fecha','finura']]
forecaster_finura = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags = _lags
             )
forecaster_finura.fit(y=datos_train['finura'])
save_pickle(forecaster_finura,'___modelo_prediccion_finura')
                  
    
#-------------- RINDE
datos_train = dataset[['fecha','rinde']]
forecaster_rinde = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags = _lags
             )
forecaster_rinde.fit(y=datos_train['rinde'])
save_pickle(forecaster_rinde,'___modelo_prediccion_rinde')


fecha            object
cant_lluvia     float64
total_ovinos      int64
kilos_lana        int64
ovejas            int64
corderoas         int64
finura          float64
rinde           float64
dtype: object
Index(['fecha', 'cant_lluvia', 'total_ovinos', 'kilos_lana', 'ovejas',
       'corderoas', 'finura', 'rinde'],
      dtype='object')
